In [1]:
from typing import Tuple

import itertools

import pandas

from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.serialize import model_to_json, model_from_json

from sklearn.metrics import mean_squared_error

/home/ankitapurv/.conda/envs/env-prophet/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
country = 'germany'

df = pandas.read_excel(io = f'../../../data/processed/{country}.xlsx')

In [3]:
def make_dataset(df_processed: pandas.DataFrame, df_covid_measures: pandas.DataFrame = pandas.DataFrame()) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_mrd = df_processed[['Time', 'Unemployment_Rate_TOT']].rename(
        columns = {'Time': 'ds', 'Unemployment_Rate_TOT': 'y'}
    )
    df_mrd = df_mrd.drop(index = df_mrd[pandas.isnull(df_mrd['y'])].index, inplace = False)
    return df_mrd, df_covid_measures

In [4]:
df_mrd, _ = make_dataset(df)

In [5]:
def train_test_split(df_mrd: pandas.DataFrame, test_size: int = 12) -> Tuple[pandas.DataFrame, pandas.DataFrame]:
    df_test = df_mrd.tail(test_size)
    df_train = df_mrd.drop(index = df_mrd.tail(test_size).index, inplace = False)
    return df_train, df_test

In [6]:
df_train, df_test = train_test_split(df_mrd, 12)

In [7]:
def train_model(df: pandas.DataFrame, param_grid: dict, eval_metric: str = 'rmse', eval_metric_optimize: str = 'min') -> Prophet:
    # Generate all combinations of parameters
    all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
    eval_metric_values = []  # Store the eval_metric_values for each params here

    # Use cross validation to evaluate all parameters
    for params in all_params:
        m = Prophet(**params).fit(df)  # Fit model with given params
        df_cv = cross_validation(m, horizon = '60 days', parallel="processes")
        df_p = performance_metrics(df_cv, rolling_window=1, monthly=True)
        eval_metric_values.append(df_p[eval_metric].values[0])

    # Find the best parameters
    tuning_results = pandas.DataFrame(all_params)
    tuning_results[eval_metric] = eval_metric_values

    tuning_results.sort_values(by = eval_metric, axis = 'index', ascending = True, inplace = True)
    best_params = {}
    if eval_metric_optimize == 'min':
        best_params = tuning_results.iloc[0].to_dict()
    elif eval_metric_optimize == 'max':
        best_params = tuning_results.iloc[-1].to_dict()
    else:
        raise ValueError("Invalid Parameter Value: param 'eval_metric_optimize' may only have values 'min' or 'max'.")


    optimal_model = Prophet(**params).fit(df)
    return optimal_model

In [8]:
param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

model = train_model(df_train, param_grid)

00:52:44 - cmdstanpy - INFO - Chain [1] start processing
00:52:44 - cmdstanpy - INFO - Chain [1] done processing
00:52:44 - cmdstanpy - ERROR - Chain [1] error: error during processing Communication error on send
Optimization terminated abnormally. Falling back to Newton.
00:52:44 - cmdstanpy - INFO - Chain [1] start processing
00:52:45 - cmdstanpy - INFO - Chain [1] done processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] start processing
00:52:46 - cmdstanpy - INFO - Chain [1] done processing
00:52:46 - cmdstanpy - INFO - Chain [1] done processing
00:52:46 - cmdstanpy - INFO - Chain [1] done p

In [9]:
def test_model(df_test: pandas.DataFrame, model: Prophet) -> Tuple[pandas.DataFrame, float]:
    "return predicted values and rmse"
    df_predicted: pandas.DataFrame = model.predict(df_test)
    rmse: float = mean_squared_error(y_true = df_test['y'], y_pred = df_predicted['yhat'], squared = False)
    return df_predicted, rmse

In [10]:
df_predicted, rmse = test_model(df_test, model)

In [11]:
df_predicted

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2022-03-01,3.233942,3.013673,3.483271,3.233942,3.233942,0.009104,0.009104,0.009104,0.009104,0.009104,0.009104,0.0,0.0,0.0,3.243046
1,2022-04-01,3.226224,3.014943,3.487744,3.226224,3.226224,0.013014,0.013014,0.013014,0.013014,0.013014,0.013014,0.0,0.0,0.0,3.239237
2,2022-05-01,3.218754,2.969695,3.452943,3.218754,3.218754,-0.007043,-0.007043,-0.007043,-0.007043,-0.007043,-0.007043,0.0,0.0,0.0,3.211711
3,2022-06-01,3.211036,2.956316,3.461009,3.211036,3.221389,-0.002443,-0.002443,-0.002443,-0.002443,-0.002443,-0.002443,0.0,0.0,0.0,3.208593
4,2022-07-01,3.203567,2.942359,3.462915,3.186932,3.241230,-0.004516,-0.004516,-0.004516,-0.004516,-0.004516,-0.004516,0.0,0.0,0.0,3.199051
5,2022-08-01,3.195849,2.942393,3.445609,3.157109,3.261641,-0.003557,-0.003557,-0.003557,-0.003557,-0.003557,-0.003557,0.0,0.0,0.0,3.192292
6,2022-09-01,3.188130,2.924435,3.462214,3.117660,3.284648,-0.000257,-0.000257,-0.000257,-0.000257,-0.000257,-0.000257,0.0,0.0,0.0,3.187873
7,2022-10-01,3.180661,2.898306,3.485564,3.063543,3.318929,-0.001913,-0.001913,-0.001913,-0.001913,-0.001913,-0.001913,0.0,0.0,0.0,3.178747
8,2022-11-01,3.172943,2.845520,3.481307,3.022323,3.374394,-0.012848,-0.012848,-0.012848,-0.012848,-0.012848,-0.012848,0.0,0.0,0.0,3.160094
9,2022-12-01,3.165473,2.812115,3.516640,2.964945,3.426598,-0.010415,-0.010415,-0.010415,-0.010415,-0.010415,-0.010415,0.0,0.0,0.0,3.155059


In [12]:
rmse

0.18460993085539482

In [13]:
with open(f'{country}_prophet_optimized_model.json', 'w') as f:
    f.write(model_to_json(model))

In [14]:
df_future = pandas.DataFrame(data = {'ds': ['2023-03-01', '2023-04-01', '2023-05-01']})

In [15]:
df_future_prediction: pandas.DataFrame = model.predict(df_future)

In [16]:
df_future_prediction

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2023-03-01,3.143065,2.921624,3.390193,3.143065,3.143065,0.009169,0.009169,0.009169,0.009169,0.009169,0.009169,0.0,0.0,0.0,3.152235
1,2023-04-01,3.135347,2.917078,3.391443,3.135347,3.135347,0.017560,0.017560,0.017560,0.017560,0.017560,0.017560,0.0,0.0,0.0,3.152907
2,2023-05-01,3.127878,2.851482,3.344162,3.127878,3.127878,-0.019030,-0.019030,-0.019030,-0.019030,-0.019030,-0.019030,0.0,0.0,0.0,3.108848
